In [46]:
import csv
import time
import sys

args = sys.argv[1:]
if len(args) == 3:
    cust, inv, inv_item = sys.argv[1:]
else:
    files = input('Input the 3 file names, separated by spaces:\n').split(' ')
    if len(files) == 3:
        cust, inv, inv_item = files
    else:
        cust, inv, inv_item = 'customer', 'invoice', 'invoice_item'
    
start_time = time.time()
# two sets to store ids and provide O(1) lookups
customer_ids = set()
invoice_ids = set()

# assume filename for sample input is 'customer_sample.csv'
# and filenames for full extraction files are 'customer.csv', 'invoice.csv', 'invoice_item.csv'
with open('customer_sample.csv') as df:
    # iterator for csv to get sample customer ids
    customer_samples = csv.reader(df)
    # skip header
    next(customer_samples)
    for row in customer_samples:
        customer_ids.add(row[0])

# function to extract data from larger files
def extract_file(filename):
    if (filename == 'invoice_item'):
        ids = invoice_ids
    else:
        ids = customer_ids

    with open('{}.csv'.format(filename)) as df1:
        # read from larger file
        data = csv.reader(df1)
        with open('{}_extract.csv'.format(filename), mode='w') as df2:
            fieldnames = next(data)
            # extract/write to smaller file
            data_extract = csv.writer(df2, quoting=csv.QUOTE_ALL)
            data_extract.writerow(fieldnames)
            # assuming that there is only one row per customer in customer.csv, keep track of ids seen
            # (there are multiple entries per customer_id/invoice_id for invoice.csv and invoice_item.csv)
            id_count = 0
            for row in data:
                # (Since the files are ASCII, I'm using standard quotes. When copying the example data 
                # from the pdf, the quote marks did not seem standard.)
                # see if customer_id (for customer & invoice) or invoice_id (for invoice_item) in ids set
                if row[0] in ids:
                    data_extract.writerow(row)
                    # get invoice_id and add to set
                    if filename == 'invoice':
                        invoice_ids.add(row[1])
                    elif filename == 'customer':
                        id_count += 1
                        # if all customer_ids seen, no need to search anymore
                        if id_count == len(customer_ids):
                            break

extract_file(cust)                   
extract_file(inv)
extract_file(inv_item)
print("--- %s seconds ---" % (time.time() - start_time))

--- 6.493710994720459 seconds ---


In [41]:
import csv
import time
import os
import sys

def get_line_at_pos(file, pos):
    file.seek(pos)
    skip = file.readline()
    npos = file.tell()
    line = file.readline()
    return npos, line

def binary_search(fname, key):
    size = os.path.getsize(fname)
    minpos, maxpos, cur = 0, size - 1, int(size / 2)
    
    with open(fname) as file:
        while (minpos <= maxpos):
            realpos, line = get_line_at_pos(file, cur)
            val = line.split(",")[0][1:-1]
            # next line just for debugging
            if key == val:
                return realpos
            if key < val:
                maxpos = cur - 1
                cur = int((minpos + maxpos) / 2)
            else:
                minpos = cur + 1
                cur = int((minpos + maxpos) / 2)
    return realpos

def extract_file(filename):
    file = '{}.csv'.format(filename)
    
    if (filename == 'invoice_item'):
        ids = invoice_ids
    else:
        ids = customer_ids
    
    with open(file) as df1:
        # read from larger file
        data = csv.reader(df1)
        with open('{}_extract.csv'.format(filename), mode='w') as df2:
            fieldnames = next(data)
            # extract/write to smaller file
            data_extract = csv.writer(df2, quoting=csv.QUOTE_ALL)
            data_extract.writerow(fieldnames)
            # assuming that there is only one row per customer in customer.csv, keep track of ids seen
            # (there are multiple entries per customer_id/invoice_id for invoice.csv and invoice_item.csv)
            id_count = 0
            for id_ in ids:
                pos = binary_search(file, id_)
                df1.seek(pos)
                row = next(data)
                # size = sys.getsizeof(row)
                data_extract.writerow(row)
                if filename == 'invoice':
                    invoice_ids.add(row[1])
                elif filename == 'customer':
                    id_count += 1
                    # if all customer_ids seen, no need to search anymore
                    if id_count == len(customer_ids):
                        break

start_time = time.time()
                        
# two sets to store ids and provide O(1) lookups
customer_ids = set()
invoice_ids = set()

# assume filename for sample input is 'customer_sample.csv'
# and filenames for full extraction files are 'customer.csv', 'invoice.csv', 'invoice_item.csv'
with open('customer_sample.csv') as df:
    # iterator for csv to get sample customer ids
    customer_samples = csv.reader(df)
    # skip header
    next(customer_samples)
    for row in customer_samples:
        customer_ids.add(row[0])
        
extract_file('customer')                   
extract_file('invoice')
extract_file('invoice_item')
print("--- %s seconds ---" % (time.time() - start_time))

--- 2.0144262313842773 seconds ---
